In [1]:
# Folium creates the map, heatmap
import folium
from folium.plugins import HeatMap, FeatureGroupSubGroup, Fullscreen, Geocoder, MiniMap
from folium.features import GeoJsonTooltip

# geopandas will read in .shp files
import geopandas as gpd
import pandas as pd
import openpyxl
import numpy as np

In [62]:
# Declare map variable
m = folium.Map(location=[34.28934316364379, -83.83165146714973],
               tiles=None,
               zoom_start=12,
               zoom_control=False,
               maxBounds=[[33.96126291577914, -84.32370348029247],[34.68922443650881, -83.22212515139717]])

# for now, comment out the Mapbox map & just include Google
# # base layers
folium.TileLayer(
    tiles = 'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1Ijoid3dyaWdodDIxIiwiYSI6ImNsNmNnbW92cjF3YXczY281NXRua25xMHgifQ.1Xa_wr0DUhuoNGP0Cbe5Kg',
    attr = 'Mapbox',
    name = 'Streets',
    overlay = False,
    control = True,
    show = True,
    min_zoom = 10,
    max_zoom = 30
    ).add_to(m)

# folium.TileLayer(
#     tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
#     attr = 'Google',
#     name = 'Google Maps',
#     overlay = False,
#     control = True,
#     show = True,
#     zoom_start = 12,
#     min_zoom = 8,
#     max_zoom = 20,
#     opacity = 1
#     ).add_to(m)

folium.TileLayer(
    tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr = 'Google',
    name = 'Satellite',
    overlay = False,
    control = True,
    min_zoom = 10,
    max_zoom = 30
    ).add_to(m)

# folium.TileLayer(
#     tiles = 'cartodbdark_matter',
#     attr = 'CartoDB',
#     name = 'Dark',
#     overlay = False,
#     control = True,
#     min_zoom = 10,
#     max_zoom = 30
#     ).add_to(m)


##################################################################################################
# ------------------------------------------------------------------------------------------------
##################################################################################################
# ------------------------------------------------------------------------------------------------
# OUTLINES!
# my_outlines = folium.map.FeatureGroup(
#     name='City / County Outline',
#     show=True,
#     control=True)

# City of Gainesville 
subject_gdf = gpd.read_file("GeoPackage/Gainesville.gpkg")
city_outline = folium.GeoJson(
    data = subject_gdf["geometry"],
    name = 'City Outline',
    show = True,
    control = True,
    style_function = lambda x: {
        'color': 'black',
        'weight': 1.5,
        'fill': False
    }).add_to(m)

# # Hall County 
# county_gdf = gpd.read_file("GeoPackage/Hall_outline.gpkg")
# county_outline = my_outlines.add_child(
#     folium.GeoJson(
#         data = county_gdf["geometry"],
#         name = 'Gainesville City Outline',
#         show = True,
#         control = True,
#         style_function = lambda x: {
#             'color': 'black',
#             'weight': 1.5,
#             'fill': False
#         }))
# m.add_child(my_outlines)


##################################################################################################
# ------------------------------------------------------------------------------------------------
##################################################################################################
# ------------------------------------------------------------------------------------------------
# school locations
data = pd.DataFrame({
    'lat':[34.303439519747464, 34.32353081469683, 34.30024514199778, 34.288963171997395, 34.248114714250846, 34.304777241900645],
    'lon':[-83.8452808576591, -83.82289012405307, -83.81358002038921, -83.86366518438354,  -83.87319396875951, -83.80822772763095],
   'name':['Centennial Arts Academy',
           'Enota Multiple Intelligences Academy',
           'Fair Street International Academy',
           'Gainesville Exploration Academy',
           'Mundy Mill Learning Academy',
           'New Holland Knowledge Academy']
}, dtype=str)

my_locations = folium.map.FeatureGroup(
    name='City School Locations',
    show=False,
    control=True)

for i in range(0,len(data)):
    my_locations.add_child(
        folium.Marker(
            location=[data.iloc[i]['lat'], data.iloc[i]['lon']],
            popup=folium.Popup(
                data.iloc[i]['name'],
                max_width=len(data.iloc[i]['name'])*20),
            icon=folium.Icon(color='green', icon_color='white', icon='graduation-cap', prefix='fa')
        ))
m.add_child(my_locations)

# Gainesville Middle (East)
my_locations.add_child(
    folium.Marker(
        location=[34.311230237360625, -83.80673528025761],
        popup=folium.Popup(
            'Gainesville Middle School (East)',
            max_width=len('Gainesville Middle School (East)')*20),
        icon=folium.Icon(color='blue', icon_color='white', icon='graduation-cap', prefix='fa')
    ))
m.add_child(my_locations)

# Gainesville Middle (West)
my_locations.add_child(
    folium.Marker(
        location=[34.280864145041974, -83.867708264793],
        popup=folium.Popup(
            'Gainesville Middle School (West)',
            max_width=len('Gainesville Middle School (West)')*20),
        icon=folium.Icon(color='blue', icon_color='white', icon='graduation-cap', prefix='fa')
    ))
m.add_child(my_locations)

# Gainesville High
my_locations.add_child(
    folium.Marker(
        location=[34.29996848425073, -83.84426178153133],
        popup=folium.Popup(
            'Gainesville High School',
            max_width=len('Gainesville High School')*20),
        icon=folium.Icon(color='red', icon_color='white', icon='graduation-cap', prefix='fa')
    ))
m.add_child(my_locations)



##################################################################################################
# ------------------------------------------------------------------------------------------------
##################################################################################################
# ------------------------------------------------------------------------------------------------
# Population
bg_gdf = gpd.read_file("GeoPackage/Gainesville_BG.gpkg")
bg_gdf = bg_gdf[['GEOID','geometry']]

# this block reads in the block groups & the population data & shmushes everything together
df1 = pd.read_csv('Tabular/2022 Total Population.csv')
df2 = pd.read_csv('Tabular/2027 Total Population.csv')

# declare year variables; this will change on or around July 1 of each year
curr_year = '2022'
proj_year = '2027'

# Data cleaning
df1['Block Group'] = df1['Block Group'].astype(str)
df1['Block Group'] = df1['Block Group'].str.replace('.', '', regex=True)
df2['Block Group'] = df2['Block Group'].astype(str)
df2['Block Group'] = df2['Block Group'].str.replace('.', '', regex=True)
df1[f'{curr_year} Total Population'] = df1[f'{curr_year} Total Population'].str.replace(',', '', regex=True).astype(int)
df2[f'{proj_year} Total Population'] = df2[f'{proj_year} Total Population'].str.replace(',', '', regex=True).astype(int)

df1.rename(columns={'Block Group': 'block_group1',
                    f'{curr_year} Total Population': f'{curr_year}_pop'},
                    inplace=True)

df2.rename(columns={'Block Group': 'block_group2',
                    f'{proj_year} Total Population': f'{proj_year}_pop'},
                    inplace=True)

# merge the current year's demographic data with the projected file
new_df = df1.merge(df2, left_on='block_group1', right_on='block_group2')
new_df.drop('block_group2', axis=1, inplace=True)
new_df.rename(columns={'block_group1':'block_group'}, inplace=True)
new_df['pop_change_asPerc'] = (new_df[f'{proj_year}_pop'] - new_df[f'{curr_year}_pop'])/new_df[f'{curr_year}_pop']
perc_labels = new_df['pop_change_asPerc']

test = bg_gdf.merge(new_df, left_on = 'GEOID', right_on = 'block_group')
test['pop_change_asPerc_2'] = test['pop_change_asPerc'].astype(float).map(lambda n: '{:.1%}'.format(n))

 # bins for the choropleth map. Shouldn't have to change, but do so with caution
custom_scale = [-2, -0.01, 0, 0.03, 0.1, 0.15, 0.25, 0.5, 1.25]

# Make the choropleth for the block group population changes
c = folium.Choropleth(
            geo_data=test,
            data=test,
            columns=['GEOID', 'pop_change_asPerc'],
            bins=custom_scale,   
            key_on='feature.properties.GEOID', 
            fill_color='RdYlGn',
            name='Population: Overall',
            fill_opacity=0.6,
            nan_fill_color="White", 
            line_opacity=0.2,
            highlight=True,
            show=False,
            line_color='black').add_to(m)

# Display Population labels when hovering
c.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = [f'{curr_year}_pop', f'{proj_year}_pop','pop_change_asPerc_2'], 
    aliases = [f'{curr_year} Population:', f'{proj_year} Population:','5-Year Projected Change:'],
    style=('background-color:grey; color:white; font-family:helvetica; font-size:12px; margin:1px'),
    localize = True,
    labels = True,
    ))

# remove the legend, which will annoyingly show up on the map when I don' wanna
for key in c._children:
    if key.startswith('color_map'):
        del(c._children[key])
c.add_to(m)


##################################################################################################
# ------------------------------------------------------------------------------------------------
##################################################################################################
# ------------------------------------------------------------------------------------------------
# Population - Ages 5-9
# this block reads in the block groups & the population data & shmushes everything together
df1 = pd.read_csv(f'Tabular/{curr_year} Population Age 5-9.csv')
df2 = pd.read_csv(f'Tabular/{proj_year} Population Age 5-9.csv')

# Data cleaning
df1['Block Group'] = df1['Block Group'].astype(str)
df1['Block Group'] = df1['Block Group'].str.replace('.', '', regex=True)
df2['Block Group'] = df2['Block Group'].astype(str)
df2['Block Group'] = df2['Block Group'].str.replace('.', '', regex=True)
# df1[f'{curr_year} Population Age 5-9'] = df1[f'{curr_year} Population Age 5-9'].str.replace(',', '', regex=True).astype(int)
# df2[f'{proj_year} Population Age 5-9'] = df2[f'{proj_year} Population Age 5-9'].str.replace(',', '', regex=True).astype(int)

df1.rename(columns={'Block Group': 'block_group1',
                    f'{curr_year} Population Age 5-9': f'{curr_year}_pop'},
                    inplace=True)

df2.rename(columns={'Block Group': 'block_group2',
                    f'{proj_year} Population Age 5-9': f'{proj_year}_pop'},
                    inplace=True)

# merge the current year's demographic data with the projected file
new_df = df1.merge(df2, left_on='block_group1', right_on='block_group2')
new_df.drop('block_group2', axis=1, inplace=True)
new_df.rename(columns={'block_group1':'block_group'}, inplace=True)
new_df['pop_change_asPerc'] = (new_df[f'{proj_year}_pop'] - new_df[f'{curr_year}_pop'])/new_df[f'{curr_year}_pop']
perc_labels = new_df['pop_change_asPerc']

test = bg_gdf.merge(new_df, left_on = 'GEOID', right_on = 'block_group')
test['pop_change_asPerc_2'] = test['pop_change_asPerc'].astype(float).map(lambda n: '{:.1%}'.format(n))

 # bins for the choropleth map. Shouldn't have to change, but do so with caution
custom_scale = [-2, -0.01, 0, 0.03, 0.1, 0.15, 0.25, 0.5, 1.25]

# Make the choropleth for the block group population changes
c_5to9 = folium.Choropleth(
            geo_data=test,
            data=test,
            columns=['GEOID', 'pop_change_asPerc'],
            bins=custom_scale,   
            key_on='feature.properties.GEOID', 
            fill_color='RdYlGn',
            name='Population: Ages 5-9',
            fill_opacity=0.6,
            nan_fill_color="White", 
            line_opacity=0.2,
            highlight=True,
            show=False,
            line_color='black').add_to(m)

# Display Population labels when hovering
c_5to9.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = [f'{curr_year}_pop', f'{proj_year}_pop','pop_change_asPerc_2'], 
    aliases = [f'{curr_year} Population:', f'{proj_year} Population:','5-Year Projected Change:'],
    style=('background-color:grey; color:white; font-family:helvetica; font-size:12px; margin:1px'),
    localize = True,
    labels = True,
    ))

# remove the legend, which will annoyingly show up on the map when I don' wanna
for key in c_5to9._children:
    if key.startswith('color_map'):
        del(c_5to9._children[key])
c_5to9.add_to(m)



##################################################################################################
# ------------------------------------------------------------------------------------------------
##################################################################################################
# ------------------------------------------------------------------------------------------------
# Population - Ages 10-14
# this block reads in the block groups & the population data & shmushes everything together
df1 = pd.read_csv(f'Tabular/{curr_year} Population Age 10-14.csv')
df2 = pd.read_csv(f'Tabular/{proj_year} Population Age 10-14.csv')

# Data cleaning
df1['Block Group'] = df1['Block Group'].astype(str)
df1['Block Group'] = df1['Block Group'].str.replace('.', '', regex=True)
df2['Block Group'] = df2['Block Group'].astype(str)
df2['Block Group'] = df2['Block Group'].str.replace('.', '', regex=True)
# df1[f'{curr_year} Population Age 5-9'] = df1[f'{curr_year} Population Age 5-9'].str.replace(',', '', regex=True).astype(int)
# df2[f'{proj_year} Population Age 5-9'] = df2[f'{proj_year} Population Age 5-9'].str.replace(',', '', regex=True).astype(int)

df1.rename(columns={'Block Group': 'block_group1',
                    f'{curr_year} Population Age 10-14': f'{curr_year}_pop'},
                    inplace=True)

df2.rename(columns={'Block Group': 'block_group2',
                    f'{proj_year} Population Age 10-14': f'{proj_year}_pop'},
                    inplace=True)

# merge the current year's demographic data with the projected file
new_df = df1.merge(df2, left_on='block_group1', right_on='block_group2')
new_df.drop('block_group2', axis=1, inplace=True)
new_df.rename(columns={'block_group1':'block_group'}, inplace=True)
new_df['pop_change_asPerc'] = (new_df[f'{proj_year}_pop'] - new_df[f'{curr_year}_pop'])/new_df[f'{curr_year}_pop']
perc_labels = new_df['pop_change_asPerc']

test = bg_gdf.merge(new_df, left_on = 'GEOID', right_on = 'block_group')
test['pop_change_asPerc_2'] = test['pop_change_asPerc'].astype(float).map(lambda n: '{:.1%}'.format(n))

 # bins for the choropleth map. Shouldn't have to change, but do so with caution
custom_scale = [-2, -0.01, 0, 0.03, 0.1, 0.15, 0.25, 0.5, 1.25]

# Make the choropleth for the block group population changes
c_10to14 = folium.Choropleth(
            geo_data=test,
            data=test,
            columns=['GEOID', 'pop_change_asPerc'],
            bins=custom_scale,   
            key_on='feature.properties.GEOID', 
            fill_color='RdYlGn',
            name='Population: Ages 10-14',
            fill_opacity=0.6,
            nan_fill_color="White", 
            line_opacity=0.2,
            highlight=True,
            show=False,
            line_color='black').add_to(m)

# Display Population labels when hovering
c_10to14.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = [f'{curr_year}_pop', f'{proj_year}_pop','pop_change_asPerc_2'], 
    aliases = [f'{curr_year} Population:', f'{proj_year} Population:','5-Year Projected Change:'],
    style=('background-color:grey; color:white; font-family:helvetica; font-size:12px; margin:1px'),
    localize = True,
    labels = True,
    ))

# remove the legend, which will annoyingly show up on the map when I don' wanna
for key in c_10to14._children:
    if key.startswith('color_map'):
        del(c_10to14._children[key])
c_10to14.add_to(m)


##################################################################################################
# ------------------------------------------------------------------------------------------------
##################################################################################################
# ------------------------------------------------------------------------------------------------
# Population - Ages 15-19
# this block reads in the block groups & the population data & shmushes everything together
df1 = pd.read_csv(f'Tabular/{curr_year} Population Age 15-19.csv')
df2 = pd.read_csv(f'Tabular/{proj_year} Population Age 15-19.csv')

# Data cleaning
df1['Block Group'] = df1['Block Group'].astype(str)
df1['Block Group'] = df1['Block Group'].str.replace('.', '', regex=True)
df2['Block Group'] = df2['Block Group'].astype(str)
df2['Block Group'] = df2['Block Group'].str.replace('.', '', regex=True)
# df1[f'{curr_year} Population Age 5-9'] = df1[f'{curr_year} Population Age 5-9'].str.replace(',', '', regex=True).astype(int)
# df2[f'{proj_year} Population Age 5-9'] = df2[f'{proj_year} Population Age 5-9'].str.replace(',', '', regex=True).astype(int)

df1.rename(columns={'Block Group': 'block_group1',
                    f'{curr_year} Population Age 15-19': f'{curr_year}_pop'},
                    inplace=True)

df2.rename(columns={'Block Group': 'block_group2',
                    f'{proj_year} Population Age 15-19': f'{proj_year}_pop'},
                    inplace=True)

# merge the current year's demographic data with the projected file
new_df = df1.merge(df2, left_on='block_group1', right_on='block_group2')
new_df.drop('block_group2', axis=1, inplace=True)
new_df.rename(columns={'block_group1':'block_group'}, inplace=True)
new_df['pop_change_asPerc'] = (new_df[f'{proj_year}_pop'] - new_df[f'{curr_year}_pop'])/new_df[f'{curr_year}_pop']
perc_labels = new_df['pop_change_asPerc']

test = bg_gdf.merge(new_df, left_on = 'GEOID', right_on = 'block_group')
test['pop_change_asPerc_2'] = test['pop_change_asPerc'].astype(float).map(lambda n: '{:.1%}'.format(n))

 # bins for the choropleth map. Shouldn't have to change, but do so with caution
custom_scale = [-2, -0.01, 0, 0.03, 0.1, 0.15, 0.25, 0.5, 1.25]

# Make the choropleth for the block group population changes
c_15to19 = folium.Choropleth(
            geo_data=test,
            data=test,
            columns=['GEOID', 'pop_change_asPerc'],
            bins=custom_scale,   
            key_on='feature.properties.GEOID', 
            fill_color='RdYlGn',
            name='Population: Ages 15-19',
            fill_opacity=0.6,
            nan_fill_color="White", 
            line_opacity=0.2,
            highlight=True,
            show=False,
            line_color='black').add_to(m)

# Display Population labels when hovering
c_15to19.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = [f'{curr_year}_pop', f'{proj_year}_pop','pop_change_asPerc_2'], 
    aliases = [f'{curr_year} Population:', f'{proj_year} Population:','5-Year Projected Change:'],
    style=('background-color:grey; color:white; font-family:helvetica; font-size:12px; margin:1px'),
    localize = True,
    labels = True,
    ))

# remove the legend, which will annoyingly show up on the map when I don' wanna
for key in c_15to19._children:
    if key.startswith('color_map'):
        del(c_15to19._children[key])
c_15to19.add_to(m)

##################################################################################################
# ------------------------------------------------------------------------------------------------
##################################################################################################
# ------------------------------------------------------------------------------------------------
# Median HH Income
df3 = pd.read_csv('Tabular/2022 Median Household Income.csv')
df3['Block Group'] = df3['Block Group'].astype(str)
df3['Block Group'] = df3['Block Group'].str.replace('.', '', regex=True)
df3.rename(columns={f'{curr_year} Median Household Income': f'{curr_year}_mHHi'}, inplace=True)
df3[f'{curr_year}_mHHi_numerical'] = df3[f'{curr_year}_mHHi'].replace('[\$,]', '', regex=True).astype(float)
test2 = bg_gdf.merge(df3, left_on = 'GEOID', right_on = 'Block Group')

# bins for the choropleth map. Shouldn't have to change, but do so with caution
custom_scale = [0, 45000, 65000, 85000, 100000, 150000, 99999999]

# Make the choropleth for the block group income levels
inc_choro = folium.Choropleth(
            geo_data=test2,
            data=test2,
            columns=['GEOID', f'{curr_year}_mHHi_numerical'],
            bins=custom_scale,   
            key_on='feature.properties.GEOID', 
            fill_color='RdYlGn',
            name='2022 Median HH Income',
            fill_opacity=0.6,
            nan_fill_color="White", 
            line_opacity=0.2,
            highlight=True,
            show=False,
            line_color='black').add_to(m)

inc_choro.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = [f'{curr_year}_mHHi', 'Block Group'], 
    aliases = [f'{curr_year} Median HH Income: ', 'Block Group ID: '],
    style=('background-color:grey; color:white; font-family:helvetica; font-size:12px; margin:1px'),
    localize = True,
    labels = True,
    ))

for key in inc_choro._children:
    if key.startswith('color_map'):
        del(inc_choro._children[key])
inc_choro.add_to(m)

##################################################################################################
# ------------------------------------------------------------------------------------------------
##################################################################################################
# ------------------------------------------------------------------------------------------------
# 2022 White Population (%)
df5 = pd.read_csv('Tabular/2022 White Population (%).csv')
df5['Block Group'] = df5['Block Group'].astype(str)
df5['Block Group'] = df5['Block Group'].str.replace('.', '', regex=True)
df5['Value'] = df5['Value'].str.replace(',', '', regex=True)
df5['Value'] = df5['Value'].astype(int)
df5.rename(columns={'Percentage of 2022 Total Population': 'pop_race_white_%'}, inplace=True)
df5['pop_race_white_%_float'] = df5['pop_race_white_%'].replace('[\%,]', '', regex=True).astype(float)
test4 = bg_gdf.merge(df5, left_on = 'GEOID', right_on = 'Block Group')

# bins for the choropleth map. Shouldn't have to change, but do so with caution
custom_scale = [0, 5, 10, 20, 30, 50, 99]

# Make the choropleth for the block group income levels
race2_choro = folium.Choropleth(
            geo_data=test4,
            data=test4,
            columns=['GEOID', 'pop_race_white_%_float'],
            bins=custom_scale,   
            key_on='feature.properties.GEOID', 
            fill_color='YlGn',
            name='2022 White Population',
            fill_opacity=0.6,
            nan_fill_color="White", 
            line_opacity=0.2,
            highlight=True,
            show=False,
            line_color='black').add_to(m)

race2_choro.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['pop_race_white_%', 'Value'], 
    aliases = ['% White Population: ', 'Total White Population: '],
    style=('background-color:grey; color:white; font-family:helvetica; font-size:12px; margin:1px'),
    localize = True,
    labels = True,
    ))

for key in race2_choro._children:
    if key.startswith('color_map'):
        del(race2_choro._children[key])
race2_choro.add_to(m)

##################################################################################################
# ------------------------------------------------------------------------------------------------
##################################################################################################
# ------------------------------------------------------------------------------------------------
# 2022 Black Population (%)
df4 = pd.read_csv('Tabular/2022 Black Population (%).csv')
df4['Block Group'] = df4['Block Group'].astype(str)
df4['Block Group'] = df4['Block Group'].str.replace('.', '', regex=True)
df4.rename(columns={'Percentage of 2022 Total Population': 'pop_race_black_%'}, inplace=True)
df4['pop_race_black_%_float'] = df4['pop_race_black_%'].replace('[\%,]', '', regex=True).astype(float)
test3 = bg_gdf.merge(df4, left_on = 'GEOID', right_on = 'Block Group')

# bins for the choropleth map. Shouldn't have to change, but do so with caution
custom_scale = [0, 5, 10, 20, 30, 50, 99]

# Make the choropleth for the block group income levels
race1_choro = folium.Choropleth(
            geo_data=test3,
            data=test3,
            columns=['GEOID', 'pop_race_black_%_float'],
            bins=custom_scale,   
            key_on='feature.properties.GEOID', 
            fill_color='YlGn',
            name='2022 Black Population',
            fill_opacity=0.6,
            nan_fill_color="White", 
            line_opacity=0.2,
            highlight=True,
            show=False,
            line_color='black').add_to(m)

race1_choro.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['pop_race_black_%', 'Value'], 
    aliases = ['% Black Population: ', 'Total Black Population: '],
    style=('background-color:grey; color:white; font-family:helvetica; font-size:12px; margin:1px'),
    localize = True,
    labels = True,
    ))

for key in race1_choro._children:
    if key.startswith('color_map'):
        del(race1_choro._children[key])
race1_choro.add_to(m)


##################################################################################################
# ------------------------------------------------------------------------------------------------
##################################################################################################
# ------------------------------------------------------------------------------------------------
# 2022 Hispanic Population (%)
df6 = pd.read_csv('Tabular/2022 Hispanic Population (%).csv')
df6['Block Group'] = df6['Block Group'].astype(str)
df6['Block Group'] = df6['Block Group'].str.replace('.', '', regex=True)
df6.rename(columns={'Percentage of 2022 Total Population': 'pop_hispanic_%'}, inplace=True)
df6['pop_hispanic_%_float'] = df6['pop_hispanic_%'].replace('[\%,]', '', regex=True).astype(float)
test5 = bg_gdf.merge(df6, left_on = 'GEOID', right_on = 'Block Group')

# bins for the choropleth map. Shouldn't have to change, but do so with caution
custom_scale = [0, 5, 10, 20, 30, 50, 99]

# Make the choropleth for the block group income levels
race3_choro = folium.Choropleth(
            geo_data=test5,
            data=test5,
            columns=['GEOID', 'pop_hispanic_%_float'],
            bins=custom_scale,   
            key_on='feature.properties.GEOID', 
            fill_color='YlGn',
            name='2022 Hispanic Population',
            fill_opacity=0.6,
            nan_fill_color="White", 
            line_opacity=0.2,
            highlight=True,
            show=False,
            line_color='black').add_to(m)

race3_choro.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['pop_hispanic_%', 'Value'], 
    aliases = ['% Hispanic Population: ', 'Total Hispanic Population: '],
    style=('background-color:grey; color:white; font-family:helvetica; font-size:12px; margin:1px'),
    localize = True,
    labels = True,
    ))

for key in race3_choro._children:
    if key.startswith('color_map'):
        del(race3_choro._children[key])
race3_choro.add_to(m)


##################################################################################################
# ------------------------------------------------------------------------------------------------
##################################################################################################
# ------------------------------------------------------------------------------------------------
# 2022 Renter Occuped HUs (%)
df7 = pd.read_csv('Tabular/2022 Renter Occupied HUs (%).csv')
df7['Block Group'] = df7['Block Group'].astype(str)
df7['Block Group'] = df7['Block Group'].str.replace('.', '', regex=True)
df7.rename(columns={'Percentage of 2022 Total Households': 'renter_HUs_%'}, inplace=True)
df7['renter_HUs_%_float'] = df7['renter_HUs_%'].replace('[\%,]', '', regex=True).astype(float)
test6 = bg_gdf.merge(df7, left_on = 'GEOID', right_on = 'Block Group')

# bins for the choropleth map. Shouldn't have to change, but do so with caution
custom_scale = [0, 5, 10, 20, 30, 50, 99]

# Make the choropleth for the block group income levels
renter_choro = folium.Choropleth(
            geo_data=test6,
            data=test6,
            columns=['GEOID', 'renter_HUs_%_float'],
            bins=custom_scale,   
            key_on='feature.properties.GEOID', 
            fill_color='YlGn',
            name='2022 Renter-Occupied Housing Units',
            fill_opacity=0.6,
            nan_fill_color="White", 
            line_opacity=0.2,
            highlight=True,
            show=False,
            line_color='black').add_to(m)

renter_choro.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['renter_HUs_%', 'Value'], 
    aliases = ['% of Renter-Occupied Housing Units: ', 'Total Renter-Occupied Units: '],
    style=('background-color:grey; color:white; font-family:helvetica; font-size:12px; margin:1px'),
    localize = True,
    labels = True,
    ))

for key in renter_choro._children:
    if key.startswith('color_map'):
        del(renter_choro._children[key])
renter_choro.add_to(m)



##################################################################################################
# ------------------------------------------------------------------------------------------------
##################################################################################################
# ------------------------------------------------------------------------------------------------
# 2022 Pop. Age 25+ w/ Bachelor's Degree (%)
df8 = pd.read_csv("Tabular/2022 Pop Age 25+ Bachelor's Degree (%).csv")
df8['Block Group'] = df8['Block Group'].astype(str)
df8['Block Group'] = df8['Block Group'].str.replace('.', '', regex=True)
df8.rename(columns={'Percentage of 2022 Educational Attainment Base': 'batch_%'}, inplace=True)
df8['batch_%_float'] = df8['batch_%'].replace('[\%,]', '', regex=True).astype(float)
test7 = bg_gdf.merge(df8, left_on = 'GEOID', right_on = 'Block Group')

# bins for the choropleth map. Shouldn't have to change, but do so with caution
custom_scale = [0, 5, 10, 20, 30, 50, 99]

# Make the choropleth for the block group income levels
batch_choro = folium.Choropleth(
            geo_data=test7,
            data=test7,
            columns=['GEOID', 'batch_%_float'],
            bins=custom_scale,   
            key_on='feature.properties.GEOID', 
            fill_color='YlGn',
            name="2022 Educational Attainment",
            fill_opacity=0.6,
            nan_fill_color="White", 
            line_opacity=0.2,
            highlight=True,
            show=False,
            line_color='black').add_to(m)

batch_choro.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['batch_%', 'Value'], 
    aliases = ["% of Pop. 25+ With Bachelor's Degree: ", 'Total Pop. 25+ With Bachelor\'s Degree: '],
    style=('background-color:grey; color:white; font-family:helvetica; font-size:12px; margin:1px'),
    localize = True,
    labels = True,
    ))

for key in batch_choro._children:
    if key.startswith('color_map'):
        del(batch_choro._children[key])
batch_choro.add_to(m)


##################################################################################################
# ------------------------------------------------------------------------------------------------
##################################################################################################
# ------------------------------------------------------------------------------------------------
# Q1 2022 RE transactions from public records
Q122_sales = pd.read_csv("Tabular/2022_HallSales_GvilleOnly.csv")
locations = list(zip(Q122_sales['lat'], Q122_sales['long']))
h1 = HeatMap(
        locations, 
        name = "2022 Residential Sales (Heat Map)",
        radius = 15,
        blur = 20,
        min_opacity = 0.75,
        overlay = True,
        show = False
        ).add_to(m)
##################################################################################################
# ------------------------------------------------------------------------------------------------
##################################################################################################
# ------------------------------------------------------------------------------------------------
# public records sales, summarize by block group
bg_gdf = gpd.read_file('GeoPackage/Gainesville_BG.gpkg')
bg_gdf = bg_gdf[['GEOID','geometry']]
# Have to cast the block group ID column as an integer to be able to join later down
bg_gdf['GEOID'] = bg_gdf['GEOID'].astype(np.int64)

# this section reads in the geocoded (and spatially joined) csv and does a bit of cleaning,
# starting with getting rid of the thousands separator and dollar sign
pub_recs = pd.read_csv('Tabular/2022_HallSales.csv')
pub_recs['saleprice_num'] = pub_recs['Sale Price'].str.replace(',', '', regex=True)
pub_recs['saleprice_num'] = pub_recs['Sale Price'].replace('[\$,]', '', regex=True).astype(float)

def Dollar(x):
    try:
        x1 = x.split('.')[0]
    except:
        x2 = np.nan
    return x1

pub_recs['Sale Price'] = pub_recs['Sale Price'].apply(lambda x: Dollar(x))

# pub_recs.drop(['field_1','Unnamed: 0','Unnamed: 0.1'], axis=1)

block_groups = pub_recs['GEOID'].unique()
count_homeSales = pub_recs.groupby('GEOID')['saleprice_num'].count()
med_homeSales = pub_recs.groupby('GEOID')['saleprice_num'].median()
max_homeSales = pub_recs.groupby('GEOID')['saleprice_num'].max()

pub_recs_BGSummary = pd.DataFrame({
    'no_of_sales':count_homeSales,
    'median_homeSale':med_homeSales,
    'max_homeSale':max_homeSales})

# this chunk of code creates a "label" columnt that will be used in the GeoJSON tooltip as a 
# nicer way to display the numerical data (will have the dollar sign and thousands separator
# added back in)
pub_recs_BGSummary['median_label'] = pub_recs_BGSummary['median_homeSale'].map("${:,.0f}".format)
pub_recs_BGSummary['max_label'] = pub_recs_BGSummary['max_homeSale'].map("${:,.0f}".format)

test_df = bg_gdf.merge(pub_recs_BGSummary, left_on = 'GEOID', right_on = 'GEOID')

# bins for the choropleth map. Shouldn't have to change, but do so with caution
custom_scale = [0, 150000, 250000, 400000, 600000, 800000]

# Make the choropleth for the block group population changes
c = folium.Choropleth(
            geo_data=test_df,
            data=test_df,
            columns=['GEOID', 'median_homeSale'],
            bins=custom_scale,   
            key_on='feature.properties.GEOID', 
            fill_color='YlGn',
            name='2022 Residential Sales (Summary)',
            fill_opacity=0.6,
            nan_fill_color="White", 
            line_opacity=0.2,
            highlight=True,
            show=False,
            line_color='black').add_to(m)

# Display Population labels when hovering
c.geojson.add_child(folium.features.GeoJsonTooltip(
    fields = ['median_label', 'no_of_sales'], 
    aliases = ['Median Home Sale Price: ', 'Total Sales: '],
    style=('background-color:grey; color:white; font-family:helvetica; font-size:12px; margin:1px'),
    localize = True,
    labels = True,
    ))

# remove the legend, which will annoyingly show up on the map when I don' wanna
for key in c._children:
    if key.startswith('color_map'):
        del(c._children[key])
c.add_to(m)

##################################################################################################
# ------------------------------------------------------------------------------------------------
##################################################################################################
# ------------------------------------------------------------------------------------------------
# Adding in layer control and all the other goodies & wonders
folium.LayerControl().add_to(m)

# add in the option to go fullscreen, which is clutch yo
var1 = Fullscreen(position='topleft',title='Full Screen').add_to(m)

# If you ever want to use a geocoder. This one is kinda bush, but it does work. Could be a good
# stopgap until I get mapbox's niiiiiice geocoder working properly
# var2 = Geocoder(collapsed=False, position='topleft', add_marker=True).add_to(m)

# This creates the minimap. Not essential by any means, but a nice flashy toy to keep the user
# interested and stimulated
var3 = MiniMap(tile_layer=None, 
               position='bottomleft', 
               width=150, 
               height=150, 
               collapsed_width=25, 
               collapsed_height=25, 
               zoom_level_offset=- 5, 
               zoom_level_fixed=None, 
               center_fixed=False, 
               zoom_animation=False, 
               toggle_display=False, 
               auto_toggle_display=False, 
               minimized=False
              ).add_to(m)

m.keep_in_front(city_outline, city_outline_w)
m.save("../index.html")
m